# Principal Component Analysis

We are going to see what the underlying dimensions of a person's performance in the decathalon are.

That is, people compete in 10 different sports in the decathalon, and receive scores for their performance in each.

Are there a smaller set of dimensions that can explain the pattern of intercorrelation between the scores on different events in the decathalon.

We are first going to start by reading in the Wikipedia table with the decathalon scores.

This is the same code we saw to read in HTML tables in the API scraping lesson

In [ ]:
import lxml.html as ET

def table_reader(source_code,table_number=0):
    doc = ET.fromstring(source_code)
    data=[]
    rows = doc.xpath("//table")[table_number].findall("tr")  
    for row in rows:
        data.append([c.text_content() for c in row.getchildren()])
    return data

url = "https://en.wikipedia.org/wiki/Athletics_at_the_2008_Summer_Olympics_%E2%80%93_Men%27s_decathlon"
content= requests.get(url)
content_string = content.text.encode('utf-8')
#content_string = content_string.replace("\n","")
data_table=table_reader(content_string,5)
decathalon = pd.DataFrame(data_table[1:],
                          index=range(len(data_table[1:])),
                          columns=data_table[0])

We want to make the event columns into a numeric format, so we are going to write a function that splits the current text on the exclamation point, and convert the text before it into a floating point number

In [ ]:
def make_numeric(content):
    try: return float(content.split("!")[0])
    except: return None
    
events = ['100 m',"LJ",'SP','HJ','400 m','110 m H','DT','PV','JT','1500 m']
for event in events:
    decathalon[event] = decathalon[event].apply(make_numeric)

Let's handle missing values by converting 0's into missing, and then drop the missing rows.

Now we have a dataframe that has all of the scores participants received for each decthalon event.

Higher scores indicate better performance

In [ ]:
decathalon[decathalon==0]=np.nan
decathalon = decathalon.dropna()
print decathalon.head(10)

We can look at the correlation between those scores to see what performance on a certain event tends to correlate with

In [ ]:
decathalon.corr()

We are going to import the principal component analysis function from sklearn and fit it to our data with differing numbers of components.

We normalize (scale) all of our columns to have a mean of 0 and standard deviation of 1 so that the relative magnitude of the variables are all similar.

We'll try components ranging from 1 to 10.

When we fit the model, we'll see how much of the explained variance is accounted for by each component.


In [ ]:
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
pc = PCA(n_components=10)
pc.fit(scale(decathalon[events]))
pc.explained_variance_

Notice that these sum to 10 (as many components that we had).


In [ ]:
np.sum(pc.explained_variance_)

This is because each variance was scaled to have a variance of 1.0

Each component explains some of the total variance.

We can look at the ratio of explained variance, to see what percentage of the total variance (i.e., 10) is accounted for by each factor

In [ ]:
pc.explained_variance_ratio_

Because we only want components that reduce the dimensionality, we will only take as many components that have an expained variance greater than 1.0

If we took a component that had an explained variance less than 1, the component would explain variance less than the original variable did individually

In [ ]:
pc = PCA(n_components=3)
pc.fit(scale(decathalon[events]))

We can use the components_ function to see how each variable correlates to each component.

These correlations between each of the variables and the components are called the "factor loadng matrix", and give you a sense of what each component represents

In [ ]:
factor_loading_matrix = pd.DataFrame(pc.components_.T,index=events)
factor_loading_matrix.sort_values(by=[0,1,2],ascending = False)

Sometimes the correlations can be hard to interpret, so a technique called factor rotation, makes the factor correlations (loadings) more straightforward

In [ ]:
def varimax_rotation(Phi, gamma = 1.0, q = 80, tol = 1e-8):
    from scipy import eye, asarray, dot, sum
    from scipy.linalg import svd
    p,k = Phi.shape
    R = eye(k)
    d=0
    for i in xrange(q):
        d_old = d
        Lambda = dot(Phi, R)
        u,s,vh = svd(dot(Phi.T,asarray(Lambda)**3 - (gamma/p) * dot(Lambda, np.diag(np.diag(dot(Lambda.T,Lambda))))))
        R = dot(u,vh)
        d = sum(s)
        if d_old!=0 and d/d_old < 1 + tol: break
    return dot(Phi, R)

In [ ]:
rotated_factors = pd.DataFrame(varimax(factor_loading_matrix),index=events)
rotated_factors.sort_values([0,1,2],ascending=False)

What are some of the large values in the rotation matrix for each column?

In [ ]:
#Factor 1: Sprinting
    
#Factor 2: Arm Strength

#Factor 3: Leg Strength
    

    

We can assign each person a facor score that describes how much that person's performance is related to each component.

In [ ]:
factor_scores = fa.transform(scale(decathalon[events]))
factor_scores_df = pd.DataFrame(factor_scores,columns=["Factor1","Factor2","Factor3"])
decathalon = pd.concat([decathalon,factor_scores_df],axis=1)

Let's look at some people who scored highest on factor 1 and what events they excelled in relative to people who scored low on factor 1

In [ ]:
print decathalon.sort_values(by="Factor1",ascending=False).head(5)
print decathalon.sort_values(by="Factor1",ascending=False).tail(5)

Because each person has a factor score for each of the 3 components, we can see how each score relates to overall performance in the decathalon, and how of much of a person's final performance can be predicted by these three components

By examining the coefficient strength, we can see what attributes best predict performance

In [ ]:
def numeric(content):
    return filter(lambda x: x in string.digits,content)
overall_score = decathalon['Overall points'].apply(numeric)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(decathalon[["Factor1","Factor2","Factor3"]],overall_score)
print lr.score(decathalon[["Factor1","Factor2","Factor3"]],overall_score)
zip(["Factor1","Factor2","Factor3"],lr.coef_)


# Linear Discriminant Analysis

Linear Discriminant Analysis is similar to PCA in that it tries to reduce the dimensions of the variables.

However, it is useful for finding the dimension that separates groups.

For example, if you have two groups of people, and they each respond to different survey questions, you can try to see what is the common underlying factor that separates them

Let's look at a survey about workplace values that were given to workers in East and West Germany.

Every person was asked to rate the importance of 13 different workplace values.

We will read in the two datasets, combine them together, and use the "key" function to keep track of which respondents came rom which area in Germany

In [ ]:
values = [
"Interesting job",
"Independent work",
"Much responsibility",
"Meaningful job",
"Chances for advancement",
"Respected job",
"Can help others",
"Useful job",
"Contact to other people",
"Secure position",
"High income",
"Much spare time",
"Healthy working condition"
]
east_germany = pd.read_csv("work%20values%20east%20germany.dat.txt",sep="\t",header=None,names=values)
west_germany = pd.read_csv("work%20values%20west%20germany.dat.txt",sep="\t",header=None,names=values)
germany_values = pd.concat([east_germany,west_germany], keys=['East', 'West'])

We can use the ix function on a dataframe to access just the rows with the key "East"

In [ ]:
germany_values.ix["East"]

Let's make a variable that has the labels for where each respondent came from

In [ ]:
location = ["East"]*len(germany_values.ix["East"])
location += ["West"]*len(germany_values.ix["West"])
print location

Now, we can provide our responses that we want to reduce into a dimmension, and their country to the LinearDiscriminant Analysis function.

We'll scale the responses to have a mean of 0 and a standard deviation of 1 before processing

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import scale

ld = LinearDiscriminantAnalysis()
ld.fit(scale(germany_values),location)

We can also assign each person a score on this dimension, with any value above 0 indicating that he/she belongs to class 1, and any values below 0 indicating that he/she belongs to class 0

In [ ]:
germany_values['Factor'] = ld.transform(scale(germany_values))
print germany_values['Factor'] 

The code below shows that any person with a factor prediction above 0 is predicted as being in "West" and any person with a factor prediction below 0 is predicted as being in "East"

In [ ]:
zip(ld.predict(scale(germany_values[values])),germany_values['Factor'])

We can examine the general accuracy of our discriminant dimension with the score function

In [ ]:
ld.score(scale(germany_values),location)

We can plot the points along one dimension and see how many of the points scoring below 0 are indeed belonging to the appropriate class and vice-versa

In [ ]:
import seaborn as sn

vector = germany_values['Factor']
n_east = len(germany_values.ix["East"])

n_west = len(germany_values.ix["West"])



import matplotlib.pyplot as plt


sn.regplot(vector[:n_west],np.array([0]*n_west),
           fit_reg=False,
           color="blue")
sn.regplot(vector[n_west:],np.array([0]*n_east),
           fit_reg=False,
           color="green")

Similarly, we can see how each of the variables relate to the Discriminant component using the factor loading matrix (which is simply the correlation of the variables to the factor)

In [ ]:
germany_values.corr()['Factor']

See how this compares to the group differences on each of the variables

In [ ]:
germany_values.groupby(location).mean()

# Cluster Analysis

In [ ]:
from sklearn.cluster import KMeans
sn.clustermap()

Cluster analysis helps identify groups of variables that are similar to other variables in that group, but different from variables in other groups.

It is usefull for understanding the different groupings of people and the natural grouping of variabes.

The following dataset contains people rank ordered preferences (from 1 to 15) for different breakfast items.

In [ ]:
breakfast_items = ["toast pop-up",
"buttered toast",
"English muffin and margarine",
"jelly donut",
"cinnamon toast",
"blueberry muffin and margarine",
"hard rolls and butter",
"toast and marmalade",
"buttered toast and jelly",
"toast and margarine",
"cinnamon bun",
"Danish pastry",
"glazed donut",
"coffee cake",
"corn muffin and butter"]

breakfast = pd.read_csv("breakfast%20items.dat.txt",sep="\t",header=None,names=breakfast_items)
breakfast

We can cluster both variables (breakfast items) and rows (people) using a heirarchical clustering method

In [ ]:
sn.clustermap(breakfast.corr(),method="complete")

We can also use a confirmatory approach to clustering where we cluster the columns by specifying the number of clusters ahead of time, and the cluster centers that minimize the distance between items in that cluster and maximize the distance between items in that cluster are found.

We can ask the algorithm to try different numbers of clusters, and choose the cluster that gives the best explanation of variance amongst the items, before we start seeing diminishing returns.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
from scipy.spatial.distance import cdist, pdist

def elbow(df, n):
    kMeansVar = [KMeans(n_clusters=k).fit(df.values) for k in range(1, n)]
    centroids = [X.cluster_centers_ for X in kMeansVar]
    k_euclid = [cdist(df.values, cent) for cent in centroids]
    dist = [np.min(ke, axis=1) for ke in k_euclid]
    wcss = [sum(d**2) for d in dist]
    tss = sum(pdist(df.values)**2)/df.values.shape[0]
    bss = tss - wcss
    return bss / tss

In [ ]:
import numpy as np
import matplotlib.pylab as P

def _line_fit(x, y):
    '''Return the slope m and intercept b of the best line fit y=mx+b to the data x[:k] and y[:k]
    for k=0..len(y).''' 
    # Figure out the m and b (in the y=mx+b sense) for the "left-of-knee"
    n, sigma_x, sigma_y, sigma_xx, sigma_xy = np.arange(1, len(y) + 1), np.cumsum(x), np.cumsum(y), np.cumsum(x * x), np.cumsum(x * y)
    det = np.maximum(n * sigma_xx - sigma_x * sigma_x, 1e-15)
    m = (n * sigma_xy - sigma_x * sigma_y) / det
    b = -(sigma_x * sigma_xy - sigma_xx * sigma_y) / det
    return m, b

'''Reverse an array.'''
_reverse = lambda x: x[-1::-1]


def knee_pt(x, y, absolute_dev=True):   
    
    # Check input args
    if len(y) < 3: raise ValueError('knee_pt: y must be at least 3 elements long')
    # Sort x, y in increasing x-order
    if any(np.diff(x) < 0):
        idx = np.argsort(x)
        x, y = x[idx], y[idx]
    else: idx = np.arange(len(x))
    
    # Regression slope (m) and intercept (b) for the "left-of-knee"
    m_fwd, b_fwd = _line_fit(x, y)
    # Regression slope (m) and intercept (b) for the "right-of-knee"
    m_bck, b_bck = map(_reverse, _line_fit(_reverse(x), _reverse(y)))
                       
    # Calculate sum of errors for left- and right- of-knee fits per point
    error_curve = np.tile(np.inf, len(y))
    for breakpt in xrange(1, len(y) - 1):
        err_fwd = m_fwd[breakpt] * x[:breakpt] + b_fwd[breakpt] - y[:breakpt]
        err_bck = m_bck[breakpt] * x[breakpt:] + b_bck[breakpt] - y[breakpt:]
        error_curve[breakpt] = sum(abs(err_fwd)) + sum(abs(err_bck)) if absolute_dev else np.sqrt(sum(err_fwd * err_fwd)) + np.sqrt(sum(err_bck * err_bck))
    # Find location of the min of the error_curve
    return idx[np.argmin(error_curve)]


def plot_knee_point(x, y, knee, text_offset=(180, -20)):
    '''Generate a plot of the data y(x), and the knee point at knee (the index of that
    point in the x- and y-arrays).'''
    idx = np.argsort(x)
    fig = P.figure()
    P.clf()
    ax = fig.add_subplot(111)
    P.hold(True)
    ax.plot(x[idx], y[idx], 'bx-', label='Data')
    P.xlabel('x')
    P.ylabel('y')
    P.plot(x[knee], y[knee], 'ro', markersize=10)
    P.annotate('Knee Point (%.3f,%.3f)' % (x[knee], y[knee]), xy=(x[knee], y[knee]), xytext=text_offset,
               textcoords='offset points', ha='right', va='bottom',
               bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
               arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
    P.title('Curve and Knee')

Let's use the above code to plot the amount of variance explained for each cluster by the number of clusters.

We want to find the knee, or where increasing the number of clusters leads to diminishing returns for the amount of variance explained

In [ ]:
%matplotlib inline
n_clusters = 10
variances_explained = elbow(breakfast,n_clusters)
factors = np.array(range(1,n_clusters))
knee = knee_pt(factors, variances_explained)
plot_knee_point(factors, variances_explained, knee)

Given that 3 clusters are suggested, we can visualize their position by plotting them against different variables

In [ ]:
kmeans = KMeans(n_clusters=3,precompute_distances=True)
kmeans.fit(breakfast)
kmeans.transform(breakfast)
cluster_assignments = pd.Series(kmeans.predict(breakfast),name="Cluster")
combined = pd.concat([breakfast,cluster_assignments],axis=1)
first_cluster = combined[(combined.Cluster == 0)]
second_cluster = combined[(combined.Cluster == 1)]
third_cluster = combined[(combined.Cluster == 2)]

sn.regplot(first_cluster['buttered toast'],first_cluster['jelly donut'],color="red",fit_reg=False)
sn.regplot(second_cluster['buttered toast'],second_cluster['jelly donut'],color="blue",fit_reg=False)
scatter = sn.regplot(third_cluster['buttered toast'],third_cluster['jelly donut'],color="green",fit_reg=False)

We can see the relative importance of each variable on the cluster by examining where the cluster is concentrated on that variable

In [ ]:
cluster_centers = pd.DataFrame([breakfast.columns,kmeans.cluster_centers_[0],
              kmeans.cluster_centers_[1],
              kmeans.cluster_centers_[2]
              ]
              ).T

cluster_centers.sort_values(by=3,ascending=False)

# Multidimensional Scaling

In [ ]:
"""
import requests
import lxml.etree as ET
import time
import itertools


url="http://www.infoplease.com/ipa/a0763098.html"
content= requests.get(url)
content_string = content.text.encode('utf-8')
content_string = content_string.replace("\n","")
data_table=table_reader(content_string,1)
cities = pd.DataFrame(data_table,index=range(len(data_table)),columns=None)[1]
cities = cities.str.replace("  ","")

pairs = itertools.combinations(cities[:15],2)

i=1
distances=[]
for pair in pairs:
    good=False
    while good != True:
        query="Distance from "+pair[0]+" to "+pair[1]
        content= requests.get("http://api.wolframalpha.com/v2/query?appid=JRUT3Q-J7X5RLLY5U&input="+query)    
        content_string = content.text.encode("utf-8")
        try: 
            doc = ET.fromstring(str(content_string))
            answer = doc.find('pod[@title="Result"]/subpod/plaintext')
        except:
            print pair
        
        if answer != None:
            result = answer.text.split(" ")[0]
            distances.append(result)
            
            good=True
            answer=None
            print i
            i+=1
        time.sleep(5)
        
from scipy.spatial.distance import squareform
distance_matrix = squareform(distances)
city_distances = pd.DataFrame(distance_matrix,index=cities[:15],columns=cities[:15])
""""

In [ ]:
city_distances = pd.read_csv("city_distances.txt")
city_distances

In [ ]:
mds = MDS(n_components = 2, dissimilarity="precomputed" ,metric=False,n_init=1000,max_iter=1000)
coordinates = mds.fit_transform(city_distances)
print coordinates
scatter = sn.regplot(coordinates[:,0], coordinates[:,1],fit_reg=False)
ax = scatter.axes
print mds.stress_
for i in range(len(city_distances)):
    ax.text(coordinates[i,0], coordinates[i,1], city_distances.columns[i])

In [ ]:
import pandas as pd
supremecourt = pd.read_csv("supremecourtdistances.txt",sep="\t")
supremecourt = supremecourt.set_index("Judge")

In [ ]:
supremecourt

Metric scaling uses the actual values of
the dissimilarities, while nonmetric scaling effectively uses only their ranks (Shepard
1962, Kruskal 1964a). Nonmetric MDS is realized by estimating an optimal monotone
transformation f(Di,j ) of the dissimilarities simultaneously with the configuration.

In [ ]:
from sklearn.manifold import MDS

mds = MDS(n_components = 1, dissimilarity="precomputed",metric=False,n_init=100,max_iter=100)
mds.fit(supremecourt)

The stress metric is how we evaluate how well our scaling represented the distances. We want the stress value to be as close to 0 as possible

In [ ]:
mds.stress_

We can get the coordinates from our dimensions for each observation to see where on the dimension the observation lies

In [ ]:
coordinates = mds.fit_transform(supremecourt)
coordinates = coordinates.flatten()
zip(supremecourt,coordinates)

We can plot these dimensions on a scatterplot.

Because we are only using one dimension, we can set the coordinates for Y to always be 0, and treat our MDS scaling as the x-axis

In [ ]:
import seaborn as sn
import numpy as np
% matplotlib inline

y_coordinates = np.array([0]*len(supremecourt))
scatter = sn.regplot(coordinates, y_coordinates,fit_reg=False)
ax = scatter.axes
ax.set_ylim(-.05,.05)
for i in range(len(supremecourt)):
    ax.text(coordinates[i], 0.025, supremecourt.columns[i],rotation=90)


## Mapping people's perceptions

In [ ]:
from sklearn.manifold import MDS

face_header = [
"Grief at death of mother",      
"Savoring a coke",      
"Very pleasant surprise",            
"Maternal love-baby in arms",       
"Physical exhaustion",               
"Something wrong with plane",       
"Anger at seeing dog beaten",        
"Pulling hard on seat of chair",    
"Unexpectedly meets old boy friend",
"Revulsion",             
"Extreme pain",                      
"Knows plane will crash",         
"Light sleep" 
]
faces = pd.read_csv("facial%20expressions.dat.txt",sep="\t",header=None, names=crime_header)
faces.index = face_header 

faces_mds = MDS(n_components=2,dissimilarity="precomputed",metric=False,n_init=100,max_iter=100)

coordinates = faces_mds.fit_transform(faces)
print faces_mds.stress_
scatter = sn.regplot(coordinates[:,0], coordinates[:,1],fit_reg=False)
ax = scatter.axes
ax.autoscale_view()
for i in range(len(crimes)):
    ax.text(coordinates[i,0], coordinates[i,1], faces.columns[i])

We can verify what the dimensions mean by having each stimulus rated on the assumed interpretation and then correlating the ratings with the coordinate values

In [ ]:
scales = pd.read_csv("facial%20expressions%20scales.dat.txt",sep="\t",header=None,names=[
        "pleasant-unpleasant", 
        "attention-rejection",
        "tension-sleep"])
print np.corrcoef(coordinates[:,1],scales['pleasant-unpleasant'])

In [ ]:
print np.corrcoef(coordinates[:,0],scales['tension-sleep'])